<a href="https://colab.research.google.com/github/dxt42250/LSTM/blob/main/LSTM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import files

uploaded = files.upload()

Saving Sentiment.csv to Sentiment.csv


In [ ]:
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM, SpatialDropout1D
from sklearn.model_selection import train_test_split, GridSearchCV
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier
import re
from keras.models import load_model
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.utils import to_categorical
# Data Preprocessing
data = pd.read_csv('Sentiment.csv')
data = data[['text', 'sentiment']]
data = data[data.sentiment != 'Neutral']

data['text'] = data['text'].apply(lambda x: x.lower())
data['text'] = data['text'].apply(lambda x: re.sub('[^a-zA-z0-9\s]', '', x))

for idx, row in data.iterrows():
    row[0] = row[0].replace('rt', ' ')

max_fatures = 2000
tokenizer = Tokenizer(num_words=max_fatures, split=' ')
tokenizer.fit_on_texts(data['text'].values)
X = tokenizer.texts_to_sequences(data['text'].values)
X = pad_sequences(X)

embed_dim = 128
lstm_out = 196

# LSTM Model Building
def create_model():
    model = Sequential()
    model.add(Embedding(max_fatures, embed_dim, input_length=X.shape[1]))
    model.add(SpatialDropout1D(0.4))
    model.add(LSTM(lstm_out, dropout=0.2, recurrent_dropout=0.2))
    model.add(Dense(2, activation='softmax'))
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

# Wrap the Keras model inside a KerasClassifier from Sci-Keras
model = KerasClassifier(build_fn=create_model, verbose=0)

# Data Splitting
labelencoder = LabelEncoder()
integer_encoded = labelencoder.fit_transform(data['sentiment'])
y = to_categorical(integer_encoded)
X_train, X_test, Y_train, Y_test = train_test_split(X, y, test_size=0.33, random_state=42)
print(X_train.shape, Y_train.shape)
print(X_test.shape, Y_test.shape)

# GridSearchCV with Sci-Keras
param_grid = {
    'batch_size': [32, 64],
    'epochs': [5, 10],
}

grid_search = GridSearchCV(estimator=model, param_grid=param_grid, cv=3)
grid_result = grid_search.fit(X_train, Y_train)

print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))

# Training the model with best hyperparameters
best_batch_size = grid_result.best_params_['batch_size']
best_epochs = grid_result.best_params_['epochs']

model = create_model()  # Create a new instance of the model with best hyperparameters
model.fit(X_train, Y_train, epochs=best_epochs, batch_size=best_batch_size, verbose=2)

# Evaluation
score, acc = model.evaluate(X_test, Y_test, verbose=2, batch_size=best_batch_size)
print("Test accuracy: %.2f%%" % (acc * 100))

# Save the model
model.save('model.h5')

# Load the model and predict on new text data
model = load_model('model.h5')
tweet = ['A lot of good things are happening. We are respected again throughout the world, and that''s a great thing.@realDonaldTrump']
tweet = tokenizer.texts_to_sequences(tweet)
tweet = pad_sequences(tweet, maxlen=X.shape[1], value=0)
print(tweet)
sentiment = model.predict(tweet, batch_size=1, verbose=1)[0]
print(sentiment)
if np.argmax(sentiment) == 0:
    print("Negative")
elif np.argmax(sentiment) == 1:
    print("Positive")


<ipython-input-3-78b1d8918642>:45: DeprecationWarning: KerasClassifier is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.
  model = KerasClassifier(build_fn=create_model, verbose=0)


(7188, 28) (7188, 2)
(3541, 28) (3541, 2)
